# GeoSpatial Analysis

This notebook intends to be an initial approach to GIS analysis in Python.

In [6]:
import pandas as pd
import geemap
import ee

In [7]:
# Initialize the library.
ee.Initialize()

In [8]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

In [10]:
# Initial date of interest (inclusive).
i_date = '2010-01-01'

# Final date of interest (exclusive).
f_date = '2022-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [11]:
# Define the urban location of interest as a point near Buenos Aires, Argentina.
u_lon = -58.381592
u_lat = -34.603722
u_poi = ee.Geometry.Point(u_lon, u_lat)

In [12]:
scale = 1000  # scale in meters

# Print the elevation near Buenos Aires, Argentina.
elv_urban_point = elv.sample(u_poi, scale).first().get('elevation').getInfo()
print('Ground elevation at urban point:', elv_urban_point, 'm')

# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

# Print the land cover type at the point.
lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
print('Land cover value at urban point is:', lc_urban_point)

Ground elevation at urban point: 40 m
Average daytime LST at urban point: 21.95 °C
Land cover value at urban point is: 13


In [13]:
# Get the data for the pixel intersecting the point in urban area.
lst_u_poi = lst.getRegion(u_poi, scale).getInfo()

# Preview the result.
lst_u_poi[:5]

[['id', 'longitude', 'latitude', 'time', 'LST_Day_1km', 'QC_Day'],
 ['2017_01_01',
  -58.3860018913483,
  -34.607596320704566,
  1483228800000,
  None,
  2],
 ['2017_01_02',
  -58.3860018913483,
  -34.607596320704566,
  1483315200000,
  15161,
  0],
 ['2017_01_03',
  -58.3860018913483,
  -34.607596320704566,
  1483401600000,
  None,
  2],
 ['2017_01_04',
  -58.3860018913483,
  -34.607596320704566,
  1483488000000,
  None,
  2]]

In [14]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [15]:
lst_df_urban = ee_array_to_df(lst_u_poi,['LST_Day_1km'])

def t_modis_to_celsius(t_modis):
    """Converts MODIS LST units to degrees Celsius."""
    t_celsius =  0.02*t_modis - 273.15
    return t_celsius

# Apply the function to get temperature in celsius.
lst_df_urban['LST_Day_1km'] = lst_df_urban['LST_Day_1km'].apply(t_modis_to_celsius)

In [17]:
lst_df_urban.tail()

,time,datetime,LST_Day_1km
1082,1576713600000,2019-12-19,34.41
1085,1576972800000,2019-12-22,30.23
1086,1577059200000,2019-12-23,31.81
1087,1577145600000,2019-12-24,33.59
1090,1577404800000,2019-12-27,31.39
